In [ ]:
# default_exp basic_model
%load_ext autoreload
%autoreload 2

In [ ]:
#export
import torch
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import GPT2Config, GPT2LMHeadModel
from kirby.run_params import RunParams
from kirby.data_manager import DataManager

# Basic Model

> A basic Pytorch lighting module

In [ ]:
#export
class BasicModel(pl.LightningModule):
    def __init__(self, run_params):
        super().__init__()
        self.run_params = run_params
        if self.run_params.pretrained:
            self.model = GPT2LMHeadModel.from_pretrained(self.run_params.model)
        else:
            config = GPT2Config()
            self.model = GPT2LMHeadModel(config)
        self.loss = torch.nn.CrossEntropyLoss(reduction='none')

    def prepare_data(self):
        data_manager = DataManager(self.run_params)
        self.train_ds, self.val_ds = data_manager.prepare_data()

    def forward(self, x):
        import pdb; pdb.set_trace()
        loss = self.model(x['input_ids'], attention_mask=x['attention_mask'], labels=x['labels'])[0]
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.forward(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self.forward(batch)
        self.log('val_loss', loss)
        return loss

    def validation_epoch_end(self, losses):
        loss = torch.cat([loss.unsqueeze(0) for loss in losses], 0).mean()
        self.log('val_loss', loss)

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
                self.train_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=True,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                ) 

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
                self.val_ds,
                batch_size=self.run_params.batch_size,
                drop_last=True,
                shuffle=False,
                num_workers=self.run_params.num_workers,
                pin_memory=True
                )

    def configure_optimizers(self):
        return torch.optim.Adam(
            self.parameters()
        ) 

# Testing

In [ ]:
# Creation
basic_model = BasicModel(RunParams())
assert isinstance(basic_model, BasicModel)

In [ ]:
run_params = RunParams(debug=True)
basic_model = BasicModel(run_params)
trainer = pl.Trainer(
    default_root_dir='logs',
    gpus=(1 if torch.cuda.is_available() else 0),
    max_epochs=run_params.max_epochs,
    fast_dev_run=run_params.debug,
    logger=TensorBoardLogger(save_dir='logs/', name=run_params.run_name),
)

trainer.fit(basic_model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).
Using custom data configuration default-d64f335cc8a13d66
Reusing dataset text (/home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
Loading cached processed dataset at /home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-afe4d9c5e6fd685f.arrow
Loading cached processed dataset at /home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-475e7db99f767f48.arrow
Loading cached processed dataset at /home/rob/.cache/huggingface/datasets/text/default-d64f335cc8a13d66/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5/cache-bc34352d2d344610.arrow
Loadin

> <ipython-input-3-5c0895eb0785>(19)forward()
     17     def forward(self, x):
     18         import pdb; pdb.set_trace()
---> 19         loss = self.model(x['input_ids'], attention_mask=x['attention_mask'], labels=x['labels'])[0]
     20         return loss
     21 



ipdb>  c


> /home/rob/miniconda3/envs/kirby/lib/python3.8/site-packages/transformers/models/gpt2/modeling_gpt2.py(969)forward()
    967             # Shift so that tokens < n predict n
    968             import pdb; pdb.set_trace()
--> 969             shift_logits = lm_logits[..., :-1, :].contiguous()
    970             shift_labels = labels[..., 1:].contiguous()
    971             # Flatten the tokens



ipdb>  labels


...

torch.Size([8, 127])


ipdb>  shift_logits.shape


torch.Size([8, 127, 50257])


ipdb>  q


BdbQuit: 

In [ ]:
basic_model.train_ds